# Assignment 1

Explain

## Olympics
---
Todo

In [19]:
import pandas as pd
df = pd.read_csv("Data/athlete_events.csv") 

## Business Understanding
---
Todo

In [4]:
1+1

2

## Data Meaning Type
---
Todo

In [5]:
1+1

2

## Data Quality
---
Todo

In [6]:
1+1

2

## Simple Statistics
---
Todo

In [7]:
1+1

2

## Visualize Attributes
---
Todo

In [8]:
1+1

2

## Explore Joint Attributes
---
Todo

In [9]:
1+1

2

## Explore Attributes and Class
---
Medal winners are going to be predicted using cross-validation against the test and train data sets. Accuracy will determine how well the prediction algorithm is performing.

## New Features
---
The data set is somewhat limited in features, so feature engineering or bringing in additional features from outside of this particular data set could be beneficial. For example, individual-specific data is limited to age, sex, weight, and height. Bringing in additional performance indicators could potentially help predict podium finishes or help with trend analysis:<br>
-Olympic best<br>
-Personal best<br>
-Body Fat %<br>
-Repeat Olympian<br>
-Previous Podium<br><br>
Bringing in additional country of origin indicators could potentially be helpful as well:<br>
-GDP per capita<br>
-Population

## Exceptional Work
---
Todo

In [22]:
1+1

2